In [37]:
Query = "Professor Kevin Chang uiuc"
questions = ["List the recent courses taught?", "from which university did the professor get their phd from", "from which university did the professor get their Bachelor from?", "What are the professor's research interests?", "what is the professor's email", "where is the professor's office","What subject does the professor teach?"]
questions

['List the recent courses taught?',
 'from which university did the professor get their phd from',
 'from which university did the professor get their Bachelor from?',
 "What are the professor's research interests?",
 "what is the professor's email",
 "where is the professor's office",
 'What subject does the professor teach?']

In [20]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import os
import re
import trafilatura as tr
import pandas as pd
import wikipedia as wiki
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import torch.nn.functional as F


In [21]:
class DocumentEmbedder:
    def __init__(self, model_name='bert-large-uncased'):
        self.model_name = model_name
        self.tokenizer =AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    def embed_doc(self, documents):
        # Initialize an empty list to store the document embeddings
        document_embeddings = []

        # Loop through each document and embed it
        for doc in documents:
            # Tokenize the document and convert it to tensors
            inputs = self.tokenizer(doc, return_tensors='pt', truncation=True, padding=True)
            
            # Forward pass through the BERT model
            with torch.no_grad():
                outputs = self.model(**inputs)
                # Extract the last layer embeddings for each token
                # embeddings = outputs.last_hidden_state
            
            # Calculate the mean of token embeddings to get document-level embedding
            doc_embedding = self.mean_pooling(outputs, inputs['attention_mask'])
            doc_embedding = F.normalize(doc_embedding)
            # Convert the PyTorch tensor to a NumPy array
            doc_embedding= doc_embedding.numpy()[0]
            document_embeddings.append(doc_embedding)

        return document_embeddings
    
    def encode_question(self, question, max_length=128):
        inputs = self.tokenizer(question, return_tensors='pt', truncation=True, padding=True)
            
        # Forward pass through the BERT model
        with torch.no_grad():
            outputs = self.model(**inputs)
            # Extract the last layer embeddings for each token
            # embeddings = outputs.last_hidden_state
        
        # Calculate the mean of token embeddings to get document-level embedding
        question_embedding = self.mean_pooling(outputs, inputs['attention_mask'])
        question_embedding = F.normalize(question_embedding)
        # Convert the PyTorch tensor to a NumPy array
        question_embedding_np= question_embedding.numpy()[0]

        return question_embedding_np

bert = DocumentEmbedder('sentence-transformers/all-MiniLM-L6-v2')

In [22]:
from serpapi import GoogleSearch

def get_query(query):
    params = {
        "q": query,
        "engine": "google",
        "api_key": "d5283d8a6c6640c36e5228ae57e8baa9170859f8b5fa73e3c941cdb51afa9e0f"
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    organic_results = results['organic_results']
    return organic_results

organic_results = get_query(Query)



In [23]:
def parse_results(link):
    downloaded = tr.fetch_url(link)
    text = tr.extract(downloaded, include_formatting=True, favor_recall=True,no_fallback=True)
    if text == None:
        return ''
    return text

def get_scholar_results(author_id):
    params = {
        'engine': 'google_scholar_author',
        'author_id': author_id,
        'api_key' : 'd5283d8a6c6640c36e5228ae57e8baa9170859f8b5fa73e3c941cdb51afa9e0f'
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    return results

dataset = {'titles': [], 'text': []}

named_entity = ""
for result in organic_results:
    if 'wikipedia.org' in result['link']:
        with open("Documents/wiki.txt", 'w') as f:
            wiki_results = wiki.search(result['title'])
            page = wiki.page(wiki_results[0], auto_suggest=False)
            f.write(page.content)
        dataset['titles'].append("wikiresult")
        dataset['text'].append(page.content)
            
    # if 'scholar.google.com' in result['link']:
    #     # get user id from link
    #     user_id = result['link'].split('user=')[1].split('&')[0]
    #     # get scholar results
    #     scholar_results = get_scholar_results(user_id)
    #     print(scholar_results["author"])
    else:
        text = parse_results(result['link'])
        if named_entity.lower() in text.lower():
            with open("Documents/" + result['title'] + '.txt', 'w') as f:
                f.write(parse_results(result['link']))
            dataset['titles'].append(result['title'])
            dataset['text'].append(parse_results(result['link']))

df = pd.DataFrame(dataset)
# df.to_csv('Dataset.csv', index=False, sep='\t')


In [24]:
# split the text into passages

def split_sections(text):
    ''' given a text split the text into sections where each section starts with some a certain number of hashtags'''

    sections = re.split(r'#+', text)
    sections = [section.strip() for section in sections if section.strip() != '']
    return sections

def clean_text(text):
    '''clean the text'''
    # Define a regular expression pattern to match punctuations and special characters except for hashtags and @ symbols
    pattern = r"[^\w\s#@.]"

    text = text.lower()
    
    # Use regex to replace the pattern with an empty string, effectively removing punctuations and special characters
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text

def split_documents(documents: dict) -> dict:
    """Split documents into passages"""
    titles, texts = [], []
    for title, text in zip(documents["titles"], documents["text"]):
        if text is not None:
            text = clean_text(text)
            for passage in split_sections(text):
                titles.append(title if title is not None else "")
                texts.append(passage)
    return {"title": titles, "text": texts}

documents = df.to_dict('list')
documents = split_documents(documents)


In [25]:
# 
documents["embedding"] = bert.embed_doc(documents["text"])
print(len(documents["embedding"]))


18


In [26]:
new_df_embed = pd.DataFrame(documents)
print(new_df_embed.shape)

(18, 3)


In [27]:
label_to_id = {label: i for i, label in enumerate(new_df_embed['title'].unique())}
print(label_to_id)
id_to_label = {i: label for label, i in label_to_id.items()}

{'Kevin Chenchuan Chang - Computer Science | UIUC': 0, 'Kevin Chen-Chuan Chang': 1, 'Kevin Chang at University Of Illinois at Urbana - Champaign': 2, 'Kevin Chen-Chuan Chang | IEEE Xplore Author Details': 3, 'Kevin Chang at the University of Illinois ...': 4, 'Kevin Chang | Department of Computer Science at Illinois': 5}


In [28]:
new_df_embed.head(-1)

,title,text,embedding
0,Kevin Chenchuan Chang - Computer Science | UIUC,kevin chenchuan chang,"[-0.021490661, 0.0062616286, 0.018235266, -0.0..."
1,Kevin Chenchuan Chang - Computer Science | UIUC,for more information,"[0.049669545, 0.0016577528, -0.08443472, -0.04..."
2,Kevin Chenchuan Chang - Computer Science | UIUC,education\n ph.d. electrial engineering stanfo...,"[-0.08992972, 0.012251538, 0.045662403, 0.0140..."
3,Kevin Chenchuan Chang - Computer Science | UIUC,biography\nkevin chenchuan chang is a professo...,"[-0.060776494, -0.00433018, 0.0028543922, -0.0..."
4,Kevin Chenchuan Chang - Computer Science | UIUC,professional highlights\n pc members sigmod vl...,"[-0.066368684, 0.057457507, -0.047124274, 0.02..."
5,Kevin Chenchuan Chang - Computer Science | UIUC,research statement\ni lead the forward data la...,"[-0.052001014, 0.045162205, 0.008414391, 0.029..."
6,Kevin Chenchuan Chang - Computer Science | UIUC,research interests\n natural language processi...,"[0.006848949, 0.0062151416, -0.00094857777, 0...."
7,Kevin Chenchuan Chang - Computer Science | UIUC,research areas,"[0.046888534, -0.015335563, -0.005797638, 0.02..."
8,Kevin Chenchuan Chang - Computer Science | UIUC,articles in conference proceedings\n are large...,"[-0.03049049, -0.06916066, 0.040925827, -0.019..."
9,Kevin Chenchuan Chang - Computer Science | UIUC,teaching honors\n uiuc list of teachers ranked...,"[-0.06295303, -0.10455318, -0.024801346, 0.008..."


In [29]:

train_features = np.array(new_df_embed['embedding'].tolist())
train_labels = np.array([label_to_id[i] for i in new_df_embed['title'].tolist()])
print("train_features shape = ", train_features.shape)
print("train_labels shape = ", train_labels.shape)

train_features shape =  (18, 384)
train_labels shape =  (18,)


In [30]:
train_features = np.array(new_df_embed['embedding'].tolist())
train_labels = np.array([i for i in range(0, len(new_df_embed))])
print("train_features shape = ", train_features.shape)
print("train_labels shape = ", train_labels.shape)

train_features shape =  (18, 384)
train_labels shape =  (18,)


In [31]:

# Create a KNN classifier with cosine similarity as the metric
knn = KNeighborsClassifier(metric='cosine')

# Fit the KNN model with document embeddings
knn.fit(train_features, train_labels) 

KNeighborsClassifier(metric='cosine')

In [32]:
question_embedding = [bert.encode_question(question) for question in questions]

In [33]:
k = 7
top_k = knn.predict_proba(question_embedding)
contexts = []
for query in top_k:
    top_k_indices = np.argsort(query)[::-1][:k]
    conatenated_context = '\n\n'.join(new_df_embed['text'].iloc[top_k_indices])
    contexts.append(conatenated_context)
    print(conatenated_context)
    print('-------------------')


coursicle  plan your perfect schedule you dont smell human... want direct access to our course data contact us.

education
 ph.d. electrial engineering stanford university 2001

recent courses taught
 cs 411  database systems
 cs 412 csp  introduction to data mining
 cs 511  advanced data management
 cs 598 kcc cs 598 kco  listening to social universe

teaching honors
 uiuc list of teachers ranked as excellent by their students fall 2001 spring 2004 fall 2005 spring 2006 fall 2010 fall 2011 fall 2019 spring 2022 spring 2023.

university of illinois at urbanachampaign

department of computer science
kcchang@illinois.edu
2114 siebel center
phone 2172442919
fax2172656494
web personal site
mail to
thomas m. siebel center for computer science
2114 siebel center
phone 2172442919
fax2172656494
web personal site
mail to
thomas m. siebel center for computer science
university of illinois mc258
201 n. goodwin avenue
urbana il 618012302
ph.d. stanford university 2001
research statement
informatio

In [34]:
# # get the top k most relevant documents
# k = 1
# top_k = knn.predict(question_embedding)

# for i in top_k:
#     print(id_to_label[i])

# # concatenate the text from the top k documents
# context = df.iloc[top_k]['text'].str.cat(sep=' ')
# # print(context)
# contexts = split_sections(context)


In [35]:
def get_answer(question, context, model_flan, tokenizer):
   
    input_text = "Answer the question as accurately as possible based on the context and say unanswerable if you can't \n\n context: v1 \n\n question: v2" 
    input_text = input_text.replace('v1', context)
    input_text = input_text.replace('v2', question)
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    outputs = model_flan.generate(input_ids,max_length=128)
    return tokenizer.decode(outputs[0])


# get the answer
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model_flan = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
for i in range(len(questions)):
    print("Question: ", questions[i])
    # print("Context: ", contexts[i])
    print(get_answer(questions[i], contexts[i], model_flan, tokenizer))


Token indices sequence length is longer than the specified maximum sequence length for this model (1216 > 512). Running this sequence through the model will result in indexing errors


Question:  List the recent courses taught?
<pad> cs 411 database systems cs 412 csp introduction to data mining cs 511 advanced data management cs 598 kcc cs 598 kco</s>
Question:  from which university did the professor get their phd from
<pad> stanford university</s>
Question:  from which university did the professor get their Bachelor from?
<pad> national taiwan university</s>
Question:  What does the professor teach?
<pad> computer science</s>
Question:  What are the professor's research interests?
<pad> information access and data management in a large scale</s>
Question:  what is the professor's email
<pad> kcchang@illinois.edu</s>
Question:  where is the professor's office
<pad> urbana il</s>
Question:  What subject does the professor teach?
<pad> computer science</s>


In [36]:
# clean the documents folder
for file in os.listdir('Documents'):
    os.remove('Documents/' + file)
